# Data Gathering & Preparation - IEMOCAP

In [592]:
import pyaudio  
import wave  
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import os

In [593]:
cwd = os.getcwd()
cwd

'C:\\Users\\sreev\\Downloads\\IEMOCAP_full_release\\code'

In [594]:
wav_dir1 = "../IEMOCAP_full_release/Session1/dialog/wav/"
wav_dir2 = "../IEMOCAP_full_release/Session2/dialog/wav/"
wav_dir3 = "../IEMOCAP_full_release/Session3/dialog/wav/"
wav_dir4 = "../IEMOCAP_full_release/Session4/dialog/wav/"
wav_dir5 = "../IEMOCAP_full_release/Session5/dialog/wav/"

wav_files = [wav_dir1,wav_dir2,wav_dir3,wav_dir4,wav_dir5]
print(wav_files)

['../IEMOCAP_full_release/Session1/dialog/wav/', '../IEMOCAP_full_release/Session2/dialog/wav/', '../IEMOCAP_full_release/Session3/dialog/wav/', '../IEMOCAP_full_release/Session4/dialog/wav/', '../IEMOCAP_full_release/Session5/dialog/wav/']


## Get wave files for each session shown above (directories)

In [595]:
wav_files_ = []
fullwav_files_=[]
for i in wav_files:
    #print(os.listdir(i))
    x = os.listdir(i)
    for j in x:
        if j.startswith("._") == False:
            wav_files_.append(j)
            fullwav_files_.append(i+j)

In [596]:
len(wav_files_)

152

## Now for each wav file, each utterence has an emotion that is recorded in text files. The below code snippet extract those text files first

In [597]:
emoeval_dir1 = "../IEMOCAP_full_release/Session1/dialog/EmoEvaluation/"
emoeval_dir2 = "../IEMOCAP_full_release/Session2/dialog/EmoEvaluation/"
emoeval_dir3 = "../IEMOCAP_full_release/Session3/dialog/EmoEvaluation/"
emoeval_dir4 = "../IEMOCAP_full_release/Session4/dialog/EmoEvaluation/"
emoeval_dir5 = "../IEMOCAP_full_release/Session5/dialog/EmoEvaluation/"
ee_files=[]
emoeval_dir = [emoeval_dir1,emoeval_dir2,emoeval_dir3,emoeval_dir4,emoeval_dir5]
emo_txt=[]
for i in emoeval_dir:
    xx = os.listdir(i)
    ee_files.append(xx)
    for j in xx:
        if j.startswith("._") == False:
            emo_txt.append(i+j)
        

In [611]:
emo_txt[0:5]

['../IEMOCAP_full_release/Session1/dialog/EmoEvaluation/.DS_Store',
 '../IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Attribute',
 '../IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Categorical',
 '../IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Self-evaluation',
 '../IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro01.txt']

## Perform the above for wave files

In [598]:
audio_emoeval_dir1 = "../IEMOCAP_full_release/Session1/sentences/wav/"
audio_emoeval_dir2 = "../IEMOCAP_full_release/Session2/sentences/wav/"
audio_emoeval_dir3 = "../IEMOCAP_full_release/Session3/sentences/wav/"
audio_emoeval_dir4 = "../IEMOCAP_full_release/Session4/sentences/wav/"
audio_emoeval_dir5 = "../IEMOCAP_full_release/Session5/sentences/wav/"
audio_ee_files=[]
audio_emoeval_dir = [audio_emoeval_dir1,audio_emoeval_dir2,audio_emoeval_dir3,audio_emoeval_dir4,audio_emoeval_dir5]
ae=[]
audio=[]
for i in audio_emoeval_dir:
    axx = os.listdir(i)
    audio.append(xx)
    for j in axx:
        if j.startswith(("._",".")) == False:
            ae.append(i+j)

In [599]:
ae[0:5]

['../IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro01',
 '../IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro02',
 '../IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro03',
 '../IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro04',
 '../IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro05']

## For wach audio wave files, extract the wav files for each utterences present in sub directories

In [604]:
audioal=[]
sub_session_audio=[]
for k in ae:
    inside = os.listdir(k)
    #print(inside)
    #audioal.append(k)
    for l in inside:
        if l.startswith(("._",".")) == False:
            sub_session_audio.append(k+"/"+l)
        

In [605]:
len(sub_session_audio)

10043

In [608]:
## Extract only text files with absolute path
emoval_txt_files=[]
txt_files = [file for file in emo_txt if file.lower().endswith('.txt')]
for i in txt_files:
    if i.startswith("._") == False:
        emoval_txt_files.append(i)
print("Text files:", emoval_txt_files[0])

Text files: ../IEMOCAP_full_release/Session1/dialog/EmoEvaluation/Ses01F_impro01.txt


## Given the text file, it contains the label that is the emotion associated with the utterence

In [609]:
def read_extract_info(file_path):
    li_content = []
    with open(file_path, 'r') as file:
        file_content = file.readlines()
        li_content.append(file_content)
        #print(li_content)
        return li_content

In [610]:
def process_result_dict(session_info, li_content):
    dict_emos = {}

    for content in li_content[0]:    
        if "\t" in content:
            splits = content.split("\t")
            for split in splits:
                if session_info in split:                
                    dict_emos[split] = splits[splits.index(split)+1]
                    #print(dict_emos)
    return dict_emos

In [612]:
# Iterate over each text file
each_txt_dict={}
all_emo_dict={}
for file_name in emoval_txt_files:
    #print(file_name)
    #extract only the name of the file
    session_info = file_name.split("/")[-1].replace(".txt","")
    #extract content of nth file
    emotion_info = read_extract_info(file_name)
    #print('----------')
    each_txt_dict[file_name] = emotion_info
    #dict_emos_al[i] = process_result_dict(session_info, result_dict[filename])
    for content in emotion_info[0]:    
        if "\t" in content:
            splits = content.split("\t")
            for split in splits:
                if session_info in split:                
                    all_emo_dict[split] = splits[splits.index(split)+1]


In [623]:
all_emo_dict

{'Ses01F_impro01_F000': 'neu',
 'Ses01F_impro01_F001': 'neu',
 'Ses01F_impro01_F002': 'neu',
 'Ses01F_impro01_F003': 'xxx',
 'Ses01F_impro01_F004': 'xxx',
 'Ses01F_impro01_F005': 'neu',
 'Ses01F_impro01_F006': 'fru',
 'Ses01F_impro01_F007': 'fru',
 'Ses01F_impro01_F008': 'fru',
 'Ses01F_impro01_F009': 'fru',
 'Ses01F_impro01_F010': 'xxx',
 'Ses01F_impro01_F011': 'fru',
 'Ses01F_impro01_F012': 'ang',
 'Ses01F_impro01_F013': 'fru',
 'Ses01F_impro01_F014': 'neu',
 'Ses01F_impro01_F015': 'fru',
 'Ses01F_impro01_M000': 'fru',
 'Ses01F_impro01_M001': 'fru',
 'Ses01F_impro01_M002': 'fru',
 'Ses01F_impro01_M003': 'fru',
 'Ses01F_impro01_M004': 'fru',
 'Ses01F_impro01_M005': 'fru',
 'Ses01F_impro01_M006': 'fru',
 'Ses01F_impro01_M007': 'fru',
 'Ses01F_impro01_M008': 'fru',
 'Ses01F_impro01_M009': 'fru',
 'Ses01F_impro01_M010': 'fru',
 'Ses01F_impro01_M011': 'ang',
 'Ses01F_impro01_M012': 'xxx',
 'Ses01F_impro01_M013': 'ang',
 'Ses01F_impro02_F000': 'sad',
 'Ses01F_impro02_F001': 'sad',
 'Ses01F

In [624]:
np.save('emotion_dictionary_formatv1.npy',all_emo_dict)

In [619]:
np.save('All_audio_sub_files.npy',sub_session_audio)

In [625]:
########################################################END#################################################################